In [1]:
import torch
from torchvision import transforms, models
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import argparse
import os
from datetime import datetime
import shutil
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import cv2

from utils_logging import setup_logger

In [2]:
from models.shashimal3 import Shashimal3
from models.__init__ import save_checkpoint, resume_checkpoint
from dataloader.shashimal2 import GooDataset
from dataloader import chong_imutils
from training.train_shashimal2 import train, test, GazeOptimizer

/home/shashimal/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [3]:
logger = setup_logger(name='first_logger',
                      log_dir ='./logs/',
                      log_file='train_chong_gooreal.log',
                      log_format = '%(asctime)s %(levelname)s %(message)s',
                      verbose=True)

In [4]:
batch_size=4
workers=4

images_dir = '/home/shashimal/Desktop/gooreal/finalrealdatasetImgsV2/'
pickle_path = '/home/shashimal/Desktop/gooreal/oneshotrealhumansNew.pickle'
test_images_dir = '/home/shashimal/Desktop/gooreal/finalrealdatasetImgsV2/'
test_pickle_path = '/home/shashimal/Desktop/gooreal/testrealhumansNew.pickle'
train_set = GooDataset(images_dir, pickle_path, 'train')
train_data_loader = DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=4)

test_set = GooDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = DataLoader(test_set, batch_size=batch_size//2,
                            shuffle=False, num_workers=4)

Number of Images: 2450
Number of Images: 2146


In [33]:
shifted_targets =  shifted_targets.cuda()

In [37]:
outputs, raw_hm = model_ft.raw_hm(img.cuda(),face.cuda(),head_channel.cuda(),object_channel.cuda())

In [45]:
import matplotlib.pyplot as plt
import numpy as np
idxs = raw_hm[0].detach().cpu().numpy()
plt.imshow(idxs, cmap='hot', interpolation='nearest')
plt.show()

/tmp/ipykernel_6855/2218337608.py:5: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [27]:
total_loss

tensor(0., dtype=torch.float64)

In [5]:
model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)


Using cache found in /home/shashimal/.cache/torch/hub/intel-isl_MiDaS_master
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Loading weights:  None


Using cache found in /home/shashimal/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


In [6]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = Shashimal3(midas)


model_ft = model_ft.to(device)

criterion = nn.NLLLoss().cuda()

# Observe that all parameters are being optimized
start_epoch = 0
max_epoch = 5
learning_rate = 1e-4

# Initializes Optimizer
gaze_opt = GazeOptimizer(model_ft, learning_rate)
optimizer = gaze_opt.getOptimizer(start_epoch)

26


In [8]:
model_ft = train(model_ft,train_data_loader, criterion, optimizer, logger,
                       num_epochs=5)

  0%|          | 0/613 [00:00<?, ?it/s]/home/shashimal/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 613/613 [30:35<00:00,  2.99s/it]


In [10]:
# torch.save(model_ft.state_dict(), 'shashimal2.pth')

In [5]:
shashimal_model = Shashimal2()
shashimal_model.load_state_dict(torch.load('shashimal2.pth'))
shashimal_model.cuda()

26


Shashimal2(
  (relu): ReLU(inplace=True)
  (sigmoid): Sigmoid()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=7, stride=1, padding=0)
  (conv1_scene): Conv2d(5, 3, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn1_scene): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (compress_conv0): Conv2d(4096, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (compress_bn0): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (compress_conv1): Conv2d(2048, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (compress_bn1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (compress_conv2): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (compress_bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (compress_conv3): Conv2d(512, 256, kernel_size=(1, 1), s

In [9]:
test(model_ft, test_data_loader, logger, save_output=True)

100%|██████████| 1073/1073 [11:29<00:00,  1.56it/s]
proximate accuracy: [4.65983225e-04 1.67753961e-02 4.05405405e-02 1.68219944e-01
 3.58341100e-01 5.31686859e-01 6.72879776e-01 7.67940354e-01]
average error: [0.8465024531794706, 0.21798499301396174, 40.50110427978448]


[0.8465024531794706, 0.21798499301396174, 40.50110427978448]

In [ ]:
img = torch.reshape(output[0],(64,64))
img.shape
writer.add_image('image',output[1])
writer.close()

In [ ]:
m = nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3, dilation=1, groups=1, bias=True)
p = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
input = torch.randn(4, 3, 448, 448)
output = p(m(input))
print(input.shape)
print(output.shape)

torch.Size([4, 3, 448, 448])
torch.Size([4, 64, 448, 448])
